In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import numbers
import hdf5storage
import scipy.io as sio

from FSSLibrary import FFTBasic as fft
import skyboxdatapy as skb

## Functions

In [ ]:
def process_skybox_case(
        dfListRow,         
        fSampling,
        pathPattern="../data_nosync/*/Measure_XLS/",
        data_sheet_names = [
            ("MP3Filtered","Filtered"), 
            ("MP3RawValue","RawValue")],
        defaultData='MP3RawValue',        
        calibTestName=None,
        calib_sheet_names = ["ConversionFactor", "ConversionOffset"]):
    
    """
    Process a Skybox test case by reading measurement and calibration data from Excel files.
        
    Parameters
    ----------
    dfListRow : pandas.Series
        A row from the test list excel sheet containing test case metadata
    pathPattern : str, optional
        Path pattern to Excel files directory 
        (default: "../data_nosync/*/Measure_XLS/")
    data_sheet_names : list of tuple, optional
        List of tuples containing (save_name, sheet_name) to read from measurement files 
        (default: [("MP3Filtered", "Filtered"), ("MP3RawValue", "RawValue")])
    defaultData : str, optional
        Name of the default dataset to use 
        (default: 'MP3RawValue')
    calibTestName : str, optional
        Name of the calibration test case to read 
        (default: None)
    calib_sheet_names : list of str, optional
        List of sheet names to read from calibration files 
        (default: ["ConversionFactor", "ConversionOffset"])
        
    Returns
    -------
    None
        Saves processed data to a .mat file in the Measure_MAT directory
    """

    print()
    
    sheets = {}

    # =====================================================================
    # Read Calibration Sheets
    if(calibTestName):
        
        calibFile = skb.io.find_unique_file(pathPattern, calibTestName, extension="xlsx")
        print(f"Processing Calibration file:\n{calibFile}")    
          
        for sheet_name in calib_sheet_names:

            df = pd.read_excel(calibFile, sheet_name=sheet_name, nrows=5)    
            df = df.drop(
                columns=['Index','Date-Time','Time','Counter','LED-chan100'], 
                errors='ignore'
            )    
            
            inner = {}
            for col in df.columns:
                arr = df[col][0] # Taking only first row
                if isinstance(arr, (np.integer, int)):
                    arr = np.float64(arr)
                inner[col] = arr
            sheets[sheet_name] = inner                


    # =====================================================================
    # Read Data Sheets
    caseName = dfListRow['C2']
    caseFile = skb.io.find_unique_file(pathPattern, caseName, extension="xlsx")
    print(f"Processing Case file:\n{caseFile}")

    for (sheet_save_name, sheet_name) in data_sheet_names:

        df = pd.read_excel(caseFile, sheet_name=sheet_name)
        df = df.drop(
            columns=['Index','Date-Time', 'Counter'], 
            errors='ignore'
        )
        
        inner = {}
        for col in df.columns:
            arr = df[col].to_numpy()        
            inner[col] = arr
        sheets[sheet_save_name] = inner

    
    # =====================================================================
    # Make default dataset choice
    defDa = sheets[defaultData].copy()
    defDa.update({"reference": defaultData})
    sheets['DefaultData'] = defDa
    


    # =====================================================================
    # Wave properties
    sheets['TestProperties'] = {
        'airGapAtMPL': dfListRow['C12']/1000,  # meters
        'calibrationFile': calibTestName if calibTestName else 'None',
        'depthAtMPL': (dfListRow['C7']-33)/1000,    # meters
        'depthAtWM': dfListRow['C7']/1000,    # meters
        'focusingLocation': dfListRow['C11'],
        'fSampling': fSampling,              # Hz
        'repeatType': dfListRow['C4'],
        'testName': dfListRow['C2'],
        'testType': dfListRow['C3'],
        'useTest': dfListRow['C6'],
        'waveAmplitude': dfListRow['C9']/1000, # meters
        'wavePeriod': float(dfListRow['C10']) if isinstance(dfListRow['C10'], numbers.Number) else dfListRow['C10'],  # seconds
        'waveType': dfListRow['C8'],
        'remarks': dfListRow['C16']
    }


    # =====================================================================
    # Save to .mat file
    outFile = caseFile.replace('Measure_XLS', 'Measure_MAT').replace('.xlsx', '.mat')
    skb.io.save_hdf5_mat(outFile, sheets)



## Example open an excel sheet and see all sheet names

In [8]:
testName = "Test491"
testFile = skb.io.find_unique_file(
    "../data_nosync/*/Measure_XLS/", testName, extension="xlsx")

xl_file = pd.ExcelFile(testFile)
sheet_names = xl_file.sheet_names
print("Test file:", testFile)
print("Sheet names:", sheet_names)


Test file: ../data_nosync/d1024/Measure_XLS/Test491_PLT675_d50_FC51_H121_FCL0960_R00.xlsx
Sheet names: ['Filtered', 'RawValue', 'Voltage']


## Process one case

In [9]:
# # Test case containing calibration data
# calibTestName = "Test358"

# # Data test case
# fileList = glob.glob("../AllTest_log/Test_log_PLT.xlsx")
# print(fileList)
# dfList = pd.read_excel(fileList[0])    

# print(dfList.columns)

# index = 6
# dfListRow = dfList.iloc[index]
# print(dfListRow)

# process_skybox_case(dfListRow, fSampling=2000.0, calibTestName=calibTestName)

## Process multiple cases

In [ ]:
# Test case containing calibration data
calibTestName = "Test_d1030_Calib"

# Data test cases
fileList = glob.glob("../AllTest_log/Test_log_PLTS.xlsx")
print(fileList)
dfList = pd.read_excel(fileList[0])    

print(dfList.columns)

for index in range(3, 15):
    dfListRow = dfList.iloc[index]
    print(f"\n=== Starting {dfListRow['C2']} ===")

    process_skybox_case(
        dfListRow, 
        fSampling=2000.0,
        pathPattern="../data_nosync/*/Measure_XLS/",
        data_sheet_names = [
            ('MP3Filtered', 'Filtered'), 
            ('MP3RawValue', 'RawValue'), 
            ('MP3Voltage', 'Voltage') ],
        defaultData='MP3Filtered',
        calibTestName=calibTestName,
        calib_sheet_names = ["ConversionFactor", "ConversionOffset"]
    )
    
    print(f"=== Finished {dfListRow['C2']} ===\n")


['../AllTest_log/Test_log_PLTS.xlsx']
Index(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'C15', 'C16'],
      dtype='object')

=== Starting Test552 ===

Processing Calibration file:
../data_nosync/d1030/Measure_XLS/Test_d1030_Calib.xlsx
Processing Case file:
../data_nosync/d1030/Measure_XLS/Test552_PLT741_d53_FC92_H127_FCL0960_R00.xlsx
=== Successfully saved using hdf5storage (nested structure with additional options) ===

=== Finished Test552 ===


=== Starting Test553 ===

Processing Calibration file:
../data_nosync/d1030/Measure_XLS/Test_d1030_Calib.xlsx
Processing Case file:
../data_nosync/d1030/Measure_XLS/Test553_PLT741_d53_FC92_H127_FCL0960_R01.xlsx
=== Successfully saved using hdf5storage (nested structure with additional options) ===

=== Finished Test553 ===


=== Starting Test554 ===

Processing Calibration file:
../data_nosync/d1030/Measure_XLS/Test_d1030_Calib.xlsx
Processing Case file:
../data_nosync/d1030/Measure_XLS/Te